Imports

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import os
import glob
from typing import List


*Transformando o Data Set Sarah 3.0 para Pandas*

In [2]:
def processarLatitudeLongitudeArquivoNC(caminho: str, destino: str, latitude_alvo: xr.DataArray, longitude_alvo: xr.DataArray):
    """
    Lê um arquivo NetCDF, filtra por uma lista de coordenadas (pontos específicos)
    usando o método 'nearest' e salva o resultado em um novo arquivo NetCDF.

    Args:
        caminho (str): O caminho completo do arquivo .nc de entrada.
        destino (str): O caminho completo onde o arquivo .nc de saída será salvo (incluindo o nome do arquivo).
        latitude (List[float]): Lista de latitudes dos pontos desejados.
        longitude (List[float]): Lista de longitudes correspondentes aos pontos desejados.

    Returns:
        None: A função não retorna valor, apenas gera o arquivo no disco.
    """

    with xr.open_dataset(caminho) as data_set:
        resultado = data_set.sel(lat=latitude_alvo, lon=longitude_alvo, method='nearest')

        pasta_destino = os.path.dirname(destino)
        if pasta_destino and not os.path.exists(pasta_destino):
            return

        resultado.to_netcdf(destino)

def processarPastaNC(caminho: str, destino: str,  latitude: List[float], longitude: List[float]):
    """
    Varre uma pasta inteira buscando arquivos .nc e processa todos eles em lote.

    Args:
        caminho (str): O diretório (pasta) onde estão os arquivos originais.
        destino (str): O diretório (pasta) onde os arquivos processados serão salvos.
        latitude (List[float]): Lista de latitudes a serem filtradas em todos os arquivos.
        longitude (List[float]): Lista de longitudes a serem filtradas em todos os arquivos.
    """
    if not os.path.exists(destino):
        os.makedirs(destino)
    
    padrao = os.path.join(caminho, "*.nc")
    arquivos_nc = glob.glob(padrao)

    latitude_alvo  = xr.DataArray(latitude, dims="pontos")
    longitude_alvo = xr.DataArray(longitude, dims="pontos")
    for indice, caminho_arquivo_atual in enumerate(arquivos_nc):
        
        nome_arquivo = os.path.basename(caminho_arquivo_atual)
        
        # Cria o novo nome (ex: "saida/dados_2024_processado.nc")
        nome_novo = f"dataset_{indice + 1:05d}.nc"
        caminho_destino = os.path.join(destino, nome_novo)
        try: 
            
            if indice % 1000 == 0:
                print(f"Processando {indice + 1}/{len(arquivos_nc)}")

            processarLatitudeLongitudeArquivoNC(
                caminho=caminho_arquivo_atual,
                destino=caminho_destino,
                latitude_alvo=latitude_alvo, 
                longitude_alvo=longitude_alvo
            )
        except Exception as e:
            print(f"Erro ao processar {nome_arquivo}: {e}")

def converterNCparaCSV(caminho: str, destino: str):
    padrao = os.path.join(caminho, "*.nc")
    arquivos_nc = glob.glob(padrao)

    if not arquivos_nc:
        print("Nenhum arquivo .nc encontrado na pasta!")
        return
    tam = len(arquivos_nc)

    lista_de_tabelas = []
    for indice, arquivo in enumerate(arquivos_nc):
        if indice % 1000 == 0:
            print(f"{indice} / {tam}")
        try:
            with xr.open_dataset(arquivo) as dataset:
                dataframe_temp = dataset.to_dataframe()
                dataframe_temp = dataframe_temp.reset_index()
                lista_de_tabelas.append(dataframe_temp)

        except Exception as e:
            print(f"Erro ao ler {arquivo}: {e}")
            return
    if lista_de_tabelas:
        df_final = pd.concat(lista_de_tabelas, ignore_index=True)
        df_final.to_csv(destino, index=True, sep=';')

        return
    else:
        print("Nenhum dado foi processado.")
        return

def filtrar_ponto_com_tolerancia(df, lat_alvo, lon_alvo, tol=0.01):
    """
    Retorna apenas as linhas que dão match com a lat E lon ao mesmo tempo.
    """
    condicao_lat = np.isclose(df['lat'], lat_alvo, atol=tol)
    condicao_lon = np.isclose(df['lon'], lon_alvo, atol=tol)
    
    # O símbolo '&' significa E (tem que satisfazer as duas condições)
    return df[condicao_lat & condicao_lon]


In [4]:
lat = [
    24.072,   # Algeria Tamarasset
    11.908,   # Nigeria Borno
    9.826,    # CAR Vakaga
    9.0723,   # Nigeria Abuja
    14.773,   # Senegal Touba
    7.25,     # Nigeria Akure
    24.475,   # Egypt Mut
    14.3675,  # Senegal Fatick
    -29.186   # South Africa Northern Cape
]

# Array de Longitudes (Eixo X)
lon = [
    4.679,    # Algeria Tamarasset
    13.427,   # Nigeria Borno
    22.508,   # CAR Vakaga
    7.4913,   # Nigeria Abuja
    -15.9196, # Senegal Touba
    5.19,     # Nigeria Akure
    28.466,   # Egypt Mut
    -16.4135, # Senegal Fatick
    20.464    # South Africa Northern Cape
]

processarPastaNC("./SARAH/ORD63865", "./SARAH/filtrado", latitude=lat, longitude=lon)

converterNCparaCSV("./SARAH/filtrado", "./SARAH/dataset_sarah_SD.csv")

Processando 1/12785
Processando 1001/12785
Processando 2001/12785
Processando 3001/12785
Processando 4001/12785
Processando 5001/12785
Processando 6001/12785
Processando 7001/12785
Processando 8001/12785
Processando 9001/12785
Processando 10001/12785
Processando 11001/12785
Processando 12001/12785
0 / 12785
1000 / 12785
2000 / 12785
3000 / 12785
4000 / 12785
5000 / 12785
6000 / 12785
7000 / 12785
8000 / 12785
9000 / 12785
10000 / 12785
11000 / 12785
12000 / 12785


**Juntando as duas tabelas**

In [5]:
# 1. Carregar os arquivos (usando o separador ';')
df_dni = pd.read_csv('./SARAH/dataset_sarah_DNI.csv', sep=';')
df_sd = pd.read_csv('./SARAH/dataset_sarah_SD.csv', sep=';')

# 2. Realizar a fusão (Merge)
# Usamos 'time', 'pontos' e 'bnds' como chave para evitar duplicação incorreta
df_merged = pd.merge(
    df_dni, 
    df_sd, 
    on=['time', 'pontos', 'bnds'], 
    suffixes=('_dni', '_sd')
)

# 3. Limpeza e Engenharia de Features
# Converter coluna de tempo para datetime
df_merged['time'] = pd.to_datetime(df_merged['time'])

# Extrair Mês e Dia (importantes para sazonalidade solar)
df_merged['year']  = df_merged['time'].dt.year
df_merged['month'] = df_merged['time'].dt.month
df_merged['day']   = df_merged['time'].dt.day

# Selecionar colunas finais
# Entradas (X): SDU (Insolação), Latitude, Longitude, Mês, Dia
# Saída (y): DNI (Irradiância Direta)
cols_final = ['time', 'year', 'month', 'day', 'lat_dni', 'lon_dni', 'SDU', 'DNI']
df_final = df_merged[cols_final].dropna() # Remove linhas com valores vazios (NaN)

# Renomear para facilitar
cols_final = ['time', 'year', 'month', 'day', 'lat', 'lon', 'SDU', 'DNI']

# 4. Visualização
print("Visualização das 5 primeiras linhas dos dados processados:")
print(df_final.head())

df_final.to_csv('./SARAH/dados_processados_sarah.csv', sep=';', index=False)

# 2. Criar a tabela de referência com os dados que você passou
ref_data = {
    'Ref_Lat': [24.072, 11.908, 9.826, 9.0723, 14.773, 7.25, 24.475, 14.3675, -29.186],
    'Ref_Lon': [4.679, 13.427, 22.508, 7.4913, -15.9196, 5.19, 28.466, -16.4135, 20.464],
    'Location': [
        'Algeria_Tamarasset', 'Nigeria_Borno', 'CAR_Vakaga', 'Nigeria_Abuja',
        'Senegal_Touba', 'Nigeria_Akure', 'Egypt_Mut', 'Senegal_Fatick', 'South_Africa_Northern_Cape'
    ]
}
df_ref = pd.DataFrame(ref_data)

# 3. Função para encontrar o local mais próximo
def get_location_name(row):
    # Calcula a distância euclidiana para todos os pontos de referência
    distances = np.sqrt(
        (df_ref['Ref_Lat'] - row['lat_dni'])**2 + 
        (df_ref['Ref_Lon'] - row['lon_dni'])**2
    )
    # Pega o índice da menor distância
    nearest_idx = distances.idxmin()
    return df_ref.loc[nearest_idx, 'Location']

# 4. Aplicar a função para criar a nova coluna
# Primeiro pegamos as combinações únicas para ser mais rápido
unique_coords = df_final[['lat_dni', 'lon_dni']].drop_duplicates()
unique_coords['Location_Name'] = unique_coords.apply(get_location_name, axis=1)

# Juntamos de volta ao dataframe principal
df_mapped = pd.merge(df_final, unique_coords, on=['lat_dni', 'lon_dni'], how='left')

# Exibir resultado
print(df_mapped[['time', 'lat_dni', 'lon_dni', 'Location_Name']].head())

# Salvar
df_mapped.to_csv('./SARAH/dados_processados_com_locais.csv', sep=';', index=False)

Visualização das 5 primeiras linhas dos dados processados:
        time  year  month  day  lat_dni  lon_dni        SDU    DNI
0 1983-01-01  1983      1    1   24.075    4.675  10.033001  290.0
1 1983-01-01  1983      1    1   11.925   13.425   9.840000  327.0
2 1983-01-01  1983      1    1    9.825   22.525  10.433001  282.0
3 1983-01-01  1983      1    1    9.075    7.475   9.883000  296.0
4 1983-01-01  1983      1    1   14.775  -15.925   9.574000  322.0
        time  lat_dni  lon_dni       Location_Name
0 1983-01-01   24.075    4.675  Algeria_Tamarasset
1 1983-01-01   11.925   13.425       Nigeria_Borno
2 1983-01-01    9.825   22.525          CAR_Vakaga
3 1983-01-01    9.075    7.475       Nigeria_Abuja
4 1983-01-01   14.775  -15.925       Senegal_Touba


In [2]:
def separar_dataset_por_local(caminho_arquivo: str, coluna_local: str = 'Location_Name'):
    """
    Lê um arquivo CSV e salva arquivos separados para cada valor único
    encontrado na coluna especificada.
    """
    try:
        # 1. Carregar o dataset principal
        # O parâmetro sep=';' é importante pois seu arquivo usa ponto e vírgula
        print(f"Lendo o arquivo: {caminho_arquivo}...")
        df = pd.read_csv(caminho_arquivo, sep=';')
        
        # Verifica se a coluna existe
        if coluna_local not in df.columns:
            print(f"Erro: Coluna '{coluna_local}' não encontrada no arquivo.")
            return

        # 2. Identificar os locais únicos
        locais_unicos = df[coluna_local].unique()
        print(f"Encontrados {len(locais_unicos)} locais únicos: {locais_unicos}")

        # Criar uma pasta para os outputs (opcional, para organização)
        pasta_saida = "datasets_separados"
        os.makedirs(pasta_saida, exist_ok=True)

        # 3. Loop para filtrar e salvar cada arquivo
        for local in locais_unicos:
            # Filtra apenas as linhas desse local
            df_local = df[df[coluna_local] == local]
            
            # Limpa o nome do local para usar no arquivo (remove caracteres ruins se houver)
            # Ex: "South_Africa_Northern_Cape" -> "South_Africa_Northern_Cape"
            nome_seguro = str(local).replace(' ', '_').replace('/', '-')
            
            # Define o nome do arquivo
            nome_arquivo = f"SARAH_{nome_seguro}.csv"
            caminho_saida = os.path.join(pasta_saida, nome_arquivo)
            
            # Salva o arquivo (mantendo o separador ; para consistência)
            df_local.to_csv(caminho_saida, sep=';', index=False)
            print(f"-> Salvo: {caminho_saida} ({len(df_local)} linhas)")

        print("\nProcesso concluído com sucesso!")

    except FileNotFoundError:
        print(f"Erro: O arquivo '{caminho_arquivo}' não foi encontrado.")
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")

arquivo_entrada = "./dataset/dados_processados_com_locais.csv"
    
separar_dataset_por_local(arquivo_entrada)

Lendo o arquivo: ./dataset/dados_processados_com_locais.csv...
Encontrados 9 locais únicos: ['Algeria_Tamarasset' 'Nigeria_Borno' 'CAR_Vakaga' 'Nigeria_Abuja'
 'Senegal_Touba' 'Nigeria_Akure' 'Egypt_Mut' 'Senegal_Fatick'
 'South_Africa_Northern_Cape']
-> Salvo: datasets_separados\SARAH_Algeria_Tamarasset.csv (22830 linhas)
-> Salvo: datasets_separados\SARAH_Nigeria_Borno.csv (22834 linhas)
-> Salvo: datasets_separados\SARAH_CAR_Vakaga.csv (22844 linhas)
-> Salvo: datasets_separados\SARAH_Nigeria_Abuja.csv (22832 linhas)
-> Salvo: datasets_separados\SARAH_Senegal_Touba.csv (22810 linhas)
-> Salvo: datasets_separados\SARAH_Nigeria_Akure.csv (22826 linhas)
-> Salvo: datasets_separados\SARAH_Egypt_Mut.csv (22852 linhas)
-> Salvo: datasets_separados\SARAH_Senegal_Fatick.csv (22808 linhas)
-> Salvo: datasets_separados\SARAH_South_Africa_Northern_Cape.csv (22850 linhas)

Processo concluído com sucesso!
